In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'netflix-prize-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1636%2F792972%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240415%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240415T071841Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D90c481b789b1fe43b208d5de340a70d3570660d39f4de14489c17e29affb60775460b9f1a9c8189c5034d18d4ac7e11a5a822cc1bcab53ea945f5d41288088ccbb10b5f1aaf0c32cb8cf5508d8282602a9117cbeccec3320e8dc9dd662c52e064770a1c604e9a20214156e521b6218e79af2fbf69e8d06f58de7e318f0b3bb0caf26daf69c2f5e0dbb86646900e31727a9cfd39ec4b046e6a5f1ca8b4d64945a96cd5686ea1a91435d210f7541ff06509ca3ca9e8577abd4fe398c201bb2c059664d66d8fab7a1d9b4dbbbba4ee77e4baff2f955bfc226884a9aeb95abc6a7cb3d9fe4105f7dc8f8e56fed883520e7fb9445f4a102dd8d32ac067388be6d92e3,merged:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4776858%2F8091014%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240415%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240415T071841Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1674ad234fe455f72fccc1007e09ccc492d2fed419d4c12bff031b67a230533a6ba64572c3aaa4dbeeaf84dfbee45ebab47f5898494a18c24bd24fb30904e453e2f18a70092657c379a77a93fb1f7f91fe1c2938917d6e1150520e3446be7dd5eac861c5ff35579e1f24ea8a5ce6cf0d435439b753a560115625182740380ea62a1d23c82c741218f7afdcfbbc392e11f1457b4a37a896e91eb9814f9d610d758c6e9a9e78185036e40dba9487f3072e2612d69e4e8b7799bf07fe567b5e1b852127926a1fc521adf9f2b675c612bf7524ffd95f3be3c243bf5189792e3e8b4b87997d4f3a5f87e4ed5636d43a23add18b73d05ec1a1d3a5f72681bf933ee053,user-movie:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4779613%2F8095161%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240415%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240415T071841Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D56c1ffa9fcb51f6cd750db23672da59f06e9a406be376f951803225b71c1fef242c3f9531949bc5c062b056820ebfe3c83ee254d68be38b0d882608060fdd3ae3ca2f028747de1ced4d5b82df5066b00a9a997d329c4b8eb06d8e608e0d713222292d1ef5c6239c3f1ca1a18fd0d3ed2ff7577e524fad54771e1312f1133867714fd77b30207dcbacaa20994aec8d4e1ea8f635615fc028528cac75a96e57e1a9ad3ab75516d43571bfadd25ae8b26e661db6205da0f1730b7a9f4121c448724ee9bfc2adf260b098ba926e36d1947b7da8bfbd7dc9c31418d3ecef426906a8b21c49e2128987dfce9424e9566d0fd4a6c7992d2f62979a9f42d8013913ff6af,movie-title:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4779739%2F8095336%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240415%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240415T071841Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9c2179f4fd2574f2a7d2e4304e935d5c1891493702fd6c323f1c43a2441593ec2e9dfaf4d704c8c5155ef84507e13261b49043ad70b19dc933535454b7f496ed92295e109189e2931a53b5f74cbfd1dddbcc7fc803319b9f2aba20fdf9790143e780ad8d02b86259692aacdee1fe76ef8264890cc1182687cf9bad37c37d58a96585f50ded37b5ee284d73671aaf38951b0d96134b786d4817a80d756ac77e71962cebca9e2ef5f225e8e8e238e3db9db3627c12a774b8b01bf2b83bc1d023d2f87be7147c426115e1c25404d68fc59e4265a7e7dc77429d9f390be5ca9b5c6390039dde019eaf2e3d28dd43733a3eb357f652e91666866fdff320dcc137a360,final-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4780457%2F8096362%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240415%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240415T071842Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9c27e91c17e86f7f8428665db3dacd2f6908cf10026c9db74ad99ffc2780cdd41c1ebc5c748e7bae07e99df5983432c7a413c4212b4d349167672dd42ea86a5af12038b41ec0d4619aa772481043a1ca238fede95c805903a0dc5c30dcb5064d5bd3f288da069519dad60c29ab1f0a079cb1c67bb92414dc53ba5fda724e85cc1217fd4c3de0d09cfd6cfa8b013a0fb08d1202762cf5e30c6ad71d971f2f99132237c27bc54e540216eac1b3ff822455a0333675c8dcf63d65b312d2e099b98fe7b96df34f81ab2380f8679e0545699b38c85a7295138c97be0d40b67fffb4e254f026509c87f3c652960c16803633540a1d4d12d09c24aa974d287937cb7e85'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/netflix-prize-data/combined_data_3.txt
/kaggle/input/netflix-prize-data/movie_titles.csv
/kaggle/input/netflix-prize-data/combined_data_4.txt
/kaggle/input/netflix-prize-data/combined_data_1.txt
/kaggle/input/netflix-prize-data/README
/kaggle/input/netflix-prize-data/probe.txt
/kaggle/input/netflix-prize-data/combined_data_2.txt
/kaggle/input/netflix-prize-data/qualifying.txt
/kaggle/input/user-movie/user_movie.npz
/kaggle/input/final-data/final_data (3).csv
/kaggle/input/movie-title/movie_title.csv
/kaggle/input/merged/merged_data.csv


In [ ]:
df = pd.read_csv("/kaggle/input/merged/merged_data.csv")

In [ ]:
df.head()

,title,director,cast,listed_in,description,MovieID
0,midnight mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...","TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...,3844
1,jeans,S. Shankar,"Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi...","Comedies, International Movies, Romantic Movies",When the father of the man she loves insists t...,372
2,grown ups,Dennis Dugan,"Adam Sandler, Kevin James, Chris Rock, David S...",Comedies,Mourning the loss of their beloved junior high...,8813
3,jaws,Steven Spielberg,"Roy Scheider, Robert Shaw, Richard Dreyfuss, L...","Action & Adventure, Classic Movies, Dramas",When an insatiable great white shark terrorize...,798
4,jaws 2,Jeannot Szwarc,"Roy Scheider, Lorraine Gary, Murray Hamilton, ...","Dramas, Horror Movies, Thrillers",Four years after the last deadly shark attacks...,11171


In [ ]:
df.shape

(753, 6)

In [ ]:
df.isnull().sum()

title            0
director       127
cast            21
listed_in        0
description      0
MovieID          0
dtype: int64

# *CONTENT BASED*

## MERGED DATA

In [ ]:
df.shape

(753, 6)

In [ ]:
#replacing nan values to an empty string as replace doesn't work on float values and "NaN" is a float value
df['cast'] = df['cast'].apply(lambda x: '' if pd.isna(x) else x.replace(" ", ""))
df['listed_in'] = df['listed_in'].apply(lambda x: '' if pd.isna(x) else x.replace(" ", ""))
df['director'] = df['director'].apply(lambda x: '' if pd.isna(x) else x.replace(" ", ""))

In [ ]:
#removing space
df['listed_in'] = df['listed_in'].apply(lambda x: x.replace(" ",""))
df['cast'] = df['cast'].apply(lambda x: x.replace(" ",""))

In [ ]:
#making tags
df['tags'] = df['director']+ " " + df['cast']  + " " +  df['description']  +  " " + df['listed_in']

In [ ]:
df

,title,director,cast,listed_in,description,MovieID,tags
0,midnight mass,MikeFlanagan,"KateSiegel,ZachGilford,HamishLinklater,HenryTh...","TVDramas,TVHorror,TVMysteries",The arrival of a charismatic young priest brin...,3844,"MikeFlanagan KateSiegel,ZachGilford,HamishLink..."
1,jeans,S.Shankar,"Prashanth,AishwaryaRaiBachchan,SriLakshmi,Nassar","Comedies,InternationalMovies,RomanticMovies",When the father of the man she loves insists t...,372,"S.Shankar Prashanth,AishwaryaRaiBachchan,SriLa..."
2,grown ups,DennisDugan,"AdamSandler,KevinJames,ChrisRock,DavidSpade,Ro...",Comedies,Mourning the loss of their beloved junior high...,8813,"DennisDugan AdamSandler,KevinJames,ChrisRock,D..."
3,jaws,StevenSpielberg,"RoyScheider,RobertShaw,RichardDreyfuss,Lorrain...","Action&Adventure,ClassicMovies,Dramas",When an insatiable great white shark terrorize...,798,"StevenSpielberg RoyScheider,RobertShaw,Richard..."
4,jaws 2,JeannotSzwarc,"RoyScheider,LorraineGary,MurrayHamilton,Joseph...","Dramas,HorrorMovies,Thrillers",Four years after the last deadly shark attacks...,11171,"JeannotSzwarc RoyScheider,LorraineGary,MurrayH..."
...,...,...,...,...,...,...,...
748,who's that knocking at my door?,MartinScorsese,"ZinaBethune,HarveyKeitel,AnneCollette,LennardK...","ClassicMovies,Dramas,IndependentMovies",A woman's revelation that she was once raped s...,12744,"MartinScorsese ZinaBethune,HarveyKeitel,AnneCo..."
749,wild wild west,BarrySonnenfeld,"WillSmith,KevinKline,KennethBranagh,SalmaHayek...","Action&Adventure,Comedies,Sci-Fi&Fantasy","Armed with an ingenious arsenal, two top-notch...",4670,"BarrySonnenfeld WillSmith,KevinKline,KennethBr..."
750,willy wonka & the chocolate factory,MelStuart,"GeneWilder,JackAlbertson,PeterOstrum,RoyKinnea...","Children&FamilyMovies,ClassicMovies,Comedies",Zany Willy Wonka causes a stir when he announc...,14050,"MelStuart GeneWilder,JackAlbertson,PeterOstrum..."
751,xxx: state of the union,LeeTamahori,"IceCube,SamuelL.Jackson,WillemDafoe,ScottSpeed...",Action&Adventure,"In this action-packed follow-up to XXX, U.S. i...",5730,"LeeTamahori IceCube,SamuelL.Jackson,WillemDafo..."


In [ ]:
#making a new dataframe
new_df = df[['title','tags']]
new_df.head()

,title,tags
0,midnight mass,"MikeFlanagan KateSiegel,ZachGilford,HamishLink..."
1,jeans,"S.Shankar Prashanth,AishwaryaRaiBachchan,SriLa..."
2,grown ups,"DennisDugan AdamSandler,KevinJames,ChrisRock,D..."
3,jaws,"StevenSpielberg RoyScheider,RobertShaw,Richard..."
4,jaws 2,"JeannotSzwarc RoyScheider,LorraineGary,MurrayH..."


In [ ]:
#converting to lowercase
new_df = new_df.copy()
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

### Stemming

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    l = []
    for i in text.split():           #split the word by a comma and list banaata hai
        l.append(ps.stem(i))

    return " ".join(l)               # list se wapas ekk string separated by space

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)
new_df.head(50)

/tmp/ipykernel_33/1650388339.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


,title,tags
0,midnight mass,"mikeflanagan katesiegel,zachgilford,hamishlink..."
1,jeans,"s.shankar prashanth,aishwaryaraibachchan,srila..."
2,grown ups,"dennisdugan adamsandler,kevinjames,chrisrock,d..."
3,jaws,"stevenspielberg royscheider,robertshaw,richard..."
4,jaws 2,"jeannotszwarc royscheider,lorrainegary,murrayh..."
5,jaws 3,"joealv dennisquaid,bessarmstrong,simonmaccorki..."
6,jaws: the revenge,"josephsarg lorrainegary,lanceguest,mariovanpee..."
7,safe house,"danielespinosa denzelwashington,ryanreynolds,v..."
8,training day,"antoinefuqua denzelwashington,ethanhawke,scott..."
9,into the night,"paulineetienne,laurentcapelluto,stefanocassett..."


In [ ]:
new_df.shape

(753, 2)

In [ ]:
def count_words(row):
    return len(row.split())

# Apply the count_words function to each row of the 'tags' column
new_df['word_count'] = df['tags'].apply(count_words)
new_df['word_count'].mean()
new_df['word_count'].sum()

/tmp/ipykernel_33/2156165864.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['word_count'] = df['tags'].apply(count_words)


20400

### Vectorization

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words = 'english')
vectors = cv.fit_transform(new_df['tags']).toarray()

### Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)
similarity

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.04347826],
       [0.        , 1.        , 0.        , ..., 0.08695652, 0.        ,
        0.13043478],
       [0.        , 0.        , 1.        , ..., 0.04256283, 0.        ,
        0.        ],
       ...,
       [0.        , 0.08695652, 0.04256283, ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.04347826, 0.13043478, 0.        , ..., 0.        , 0.        ,
        1.        ]])

### Recomendation Function

In [ ]:
def recommend(movie):
    movie_index = df[df['title'] == movie].index[0]             #index fetch karenge
    similar_movies = sorted(list(enumerate(similarity[movie_index])), reverse = True, key = lambda x: x[1])[1:6]   #sabse kam distance wali movies

    for i in similar_movies:
        print(new_df.iloc[i[0]].title)


In [ ]:
recommend('a cinderella story')

good burger
chitty chitty bang bang
dennis the menace
dr. seuss' the cat in the hat
stuart little


In [ ]:
recommend("jaws")

jaws 2
jaws: the revenge
jaws 3
deep blue sea
platoon


In [ ]:
import pickle

In [ ]:
pickle.dump(new_df, open('movies.pkl', 'wb'))

In [ ]:
pickle.dump(new_df.to_dict(), open('movie_dict.pkl', 'wb'))

In [ ]:
movies_dict = new_df.to_dict()

In [ ]:
movies = pd.DataFrame(movies_dict)

In [ ]:
movies['title'].values

array(['midnight mass', 'jeans', 'grown ups', 'jaws', 'jaws 2', 'jaws 3',
       'jaws: the revenge', 'safe house', 'training day',
       'into the night', 'the circle', 'in the cut', 'a cinderella story',
       'clear and present danger', 'cliffhanger', 'cold mountain',
       'crocodile dundee in los angeles', 'do the right thing',
       'house party', 'house party 2', 'house party 3',
       'i got the hook up', 'in too deep', 'initial d', 'labyrinth',
       "love don't cost a thing", 'mars attacks!', "my boss's daughter",
       'mystery men', 'once upon a time in america', 'osmosis jones',
       'rhyme & reason', 'tears of the sun', 'the blue lagoon',
       'the golden child', 'the guns of navarone', 'the nutty professor',
       'the nutty professor ii: the klumps', 'in the line of fire',
       'wind river', 'family reunion', 'open your eyes', 'bombay',
       'the piano', 'shaman king', 'quartet', 'the swarm', 'heavy',
       'beethoven', "beethoven's 2nd", 'beowulf', 'ca